In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_ = pd.read_csv("/Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/out/MELKOM_HES_birlesik_2024.csv")
df = df_.copy()

In [3]:
print("=== Veri Kalite Kontrolü ===")
print(f"Toplam satır: {len(df)}")
print(f"Eksik değerler: {df.isnull().sum().sum()}")
print(f"Duplikat satırlar: {df.duplicated().sum()}")

# Saat kontrolü (24 saat × 366 gün = 8784)
print(f"\n=== Saat Dağılımı ===")
print(f"Benzersiz saat sayısı: {df['Saat'].nunique()}")
print(f"Benzersiz tarih sayısı: {df['Tarih'].nunique()}")

# Değer aralıkları
print(f"\n=== Değer Aralıkları ===")
print(f"PTF: {df['PTF'].min():.2f} - {df['PTF'].max():.2f}")
print(f"SMF: {df['SMF'].min():.2f} - {df['SMF'].max():.2f}")
print(f"KGÜP: {df['KGUP'].min():.2f} - {df['KGUP'].max():.2f}")
print(f"Üretim: {df['URETIM'].min():.2f} - {df['URETIM'].max():.2f}")

# Dengesizlik maliyeti özeti
print(f"\n=== Dengesizlik Maliyeti ===")
print(f"Toplam DM: {df['Dengesizlik_Maliyeti'].sum():.2f} TL")
print(f"Ortalama Birim DM: {df['Birim_DM'].mean():.4f} TL/MWh")
print(f"Toplam Üretim: {df['URETIM'].sum():.2f} MWh")

=== Veri Kalite Kontrolü ===
Toplam satır: 8775
Eksik değerler: 0
Duplikat satırlar: 0

=== Saat Dağılımı ===
Benzersiz saat sayısı: 24
Benzersiz tarih sayısı: 366

=== Değer Aralıkları ===
PTF: 0.00 - 3000.00
SMF: 0.00 - 3000.00
KGÜP: 0.00 - 7.30
Üretim: 0.00 - 7.61

=== Dengesizlik Maliyeti ===
Toplam DM: 2588310.23 TL
Ortalama Birim DM: 99.7899 TL/MWh
Toplam Üretim: 33658.29 MWh


In [3]:
df["Net_Gelir"].sum()

np.float64(72381318.94182001)

In [22]:
import calendar

# 2024 yılı için her ayın gün sayısı
for month in range(1, 13):
    days = calendar.monthrange(2024, month)[1]
    month_name = calendar.month_name[month]
    print(f"{month_name}: {days} gün")

January: 31 gün
February: 29 gün
March: 31 gün
April: 30 gün
May: 31 gün
June: 30 gün
July: 31 gün
August: 31 gün
September: 30 gün
October: 31 gün
November: 30 gün
December: 31 gün


In [7]:
# Aylık agregasyon ve temel performans metrikleri
df_monthly = df.groupby('Ay').agg({
    'KGUP': 'sum',
    'URETIM': 'sum', 
    'Dengesizlik': 'sum',
    'GOP_Geliri': 'sum',
    'Dengesizlik_Tutarı': 'sum',
    'Net_Gelir': 'sum',
    'Dengesizlik_Maliyeti': 'sum'
}).round(2)

# Aylık performans metrikleri
df_monthly['Dengesizlik_Orani'] = (df_monthly['Dengesizlik'] / df_monthly['KGUP'] * 100).round(4)
df_monthly['Birim_DM_Aylik'] = (df_monthly['Dengesizlik_Maliyeti'] / df_monthly['URETIM']).round(4)

# Kapasite faktörü hesaplama (düzeltilmiş)
days_per_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]  # 2024 artık yıl
max_possible_production = [7.3 * 24 * days for days in days_per_month]
df_monthly['Kapasite_Faktoru'] = (df_monthly['URETIM'] / max_possible_production * 100).round(2)

df_monthly['DM_Gelir_Orani'] = (df_monthly['Dengesizlik_Maliyeti'] / df_monthly['Net_Gelir'] * 100).round(4)

print("=== MELKOM HES - Aylık Performans Özeti ===")
df_monthly

=== MELKOM HES - Aylık Performans Özeti ===


,KGUP,URETIM,Dengesizlik,GOP_Geliri,Dengesizlik_Tutarı,Net_Gelir,Dengesizlik_Maliyeti,Dengesizlik_Orani,Birim_DM_Aylik,Kapasite_Faktoru,DM_Gelir_Orani
Ay,,,,,,,,,,,
1,3014.7,3216.29,201.59,5865731.78,268952.11,6134683.89,221318.41,6.6869,68.8117,59.22,3.6077
2,2369.7,2383.59,13.89,4478744.52,-137733.06,4341011.46,255104.80,0.5862,107.0255,46.91,5.8766
3,1890.8,2063.99,173.19,4070503.82,260152.46,4330656.28,172027.99,9.1596,83.3473,38.00,3.9723
4,3902.6,4016.64,114.04,7215394.25,29264.91,7244659.16,293041.19,2.9222,72.9568,76.42,4.0449
5,4772.2,4810.36,38.16,9836138.82,-89062.40,9747076.43,210122.39,0.7996,43.6812,88.57,2.1557
6,3194.4,3225.47,31.07,7002776.24,-48455.38,6954320.86,174919.56,0.9726,54.2307,61.37,2.5153
7,2397.3,2530.77,133.47,6434709.05,195311.84,6630020.89,143981.58,5.5675,56.8924,46.60,2.1717
8,1569.8,1704.98,135.18,4206366.68,133850.74,4340217.42,204851.23,8.6113,120.1488,31.39,4.7198
9,1285.3,1243.44,-41.86,3134327.98,-234750.07,2899577.90,175577.59,-3.2568,141.2031,23.66,6.0553


In [21]:
# Ceza asimetrisi stratejisi (overweight optimizasyonu)
print("=== YANBOLU HES - Ceza Asimetrisi Stratejisi (Overweight Optimizasyonu) ===")

# Pozitif ve negatif dengesizliklerin ceza oranlarını analiz et
positive_imb = df[df['Dengesizlik'] > 0].copy()
negative_imb = df[df['Dengesizlik'] < 0].copy()

if len(positive_imb) > 0 and len(negative_imb) > 0:
    # Ceza oranları (Dengesizlik Tutarı / (Dengesizlik * PTF))
    positive_imb['Penalty_Rate'] = positive_imb['Dengesizlik_Tutarı'] / (positive_imb['Dengesizlik'] * positive_imb['PTF'])
    negative_imb['Penalty_Rate'] = abs(negative_imb['Dengesizlik_Tutarı']) / (abs(negative_imb['Dengesizlik']) * negative_imb['PTF'])
    
    avg_pos_penalty = positive_imb['Penalty_Rate'].mean()
    avg_neg_penalty = negative_imb['Penalty_Rate'].mean()
    
    print(f"Ortalama Pozitif Dengesizlik Ceza Oranı: {avg_pos_penalty:.3f}")
    print(f"Ortalama Negatif Dengesizlik Ceza Oranı: {avg_neg_penalty:.3f}")
    print(f"Ceza Asimetrisi (Neg/Pos): {avg_neg_penalty/avg_pos_penalty:.2f}")
    
    # Overweight stratejisi simülasyonu
    overweight_scenarios = [0.05, 0.10, 0.15, 0.20]  # %5, %10, %15, %20 fazla planlama
    
    print(f"\nOverweight Stratejisi Simülasyonu:")
    for ow in overweight_scenarios:
        sim_kgup = df['KGUP'] * (1 + ow)
        sim_dengesizlik = df['URETIM'] - sim_kgup
        
        # Yeni dengesizlik tutarları (vectorized çözüm)
        sim_dt = np.where(
            sim_dengesizlik >= 0,
            sim_dengesizlik * df['Poz_DF'],
            sim_dengesizlik * df['Neg_DF']
        )
        
        # Dengesizlik maliyeti değişimi
        original_dm = df['Dengesizlik_Maliyeti'].sum()
        sim_dm = ((df['URETIM'] * df['PTF']) - (sim_kgup * df['PTF'] + sim_dt)).clip(lower=0).sum()
        
        dm_change = sim_dm - original_dm
        dm_change_pct = (dm_change / original_dm) * 100
        
        print(f"  {ow*100:2.0f}% Overweight: DM Değişimi = {dm_change:,.0f} TL ({dm_change_pct:+.1f}%)")

=== YANBOLU HES - Ceza Asimetrisi Stratejisi (Overweight Optimizasyonu) ===
Ortalama Pozitif Dengesizlik Ceza Oranı: 0.837
Ortalama Negatif Dengesizlik Ceza Oranı: 1.244
Ceza Asimetrisi (Neg/Pos): 1.49

Overweight Stratejisi Simülasyonu:
   5% Overweight: DM Değişimi = 59,244 TL (+2.3%)
  10% Overweight: DM Değişimi = 213,229 TL (+8.2%)
  15% Overweight: DM Değişimi = 410,414 TL (+15.9%)
  20% Overweight: DM Değişimi = 633,805 TL (+24.5%)


In [26]:
print("=== MELKOM HES - Risk Analizi ===")

# En yüksek DM yaşanan günler
daily_dm = df.groupby('Tarih')['Dengesizlik_Maliyeti'].sum()
print("En yüksek dengesizlik maliyeti yaşanan 10 gün:")
# En yüksek dengesizlik maliyeti yaşanan 10 günü, önce DM'ye göre en yüksekten seçip, sonra tarihe göre (en yeni gün en üstte) göstermek için:
top10_dm_days = daily_dm.sort_values(ascending=False).head(10)
top10_dm_days = top10_dm_days.sort_index(ascending=False)
print(top10_dm_days)

hourly_analysis = df.groupby('Saat').agg({
    'Dengesizlik_Maliyeti': 'mean',
    'Dengesizlik': 'mean',
    'PTF': 'mean',
    'KGUP': 'mean'
}).reset_index()

# Dengesizlik oranı = dengesizlik / planlanan üretim
hourly_analysis['Dengesizlik_Orani'] = (
    hourly_analysis['Dengesizlik'] / hourly_analysis['KGUP'] * 100
).round(2)

# En riskli saatler
risky_hours = hourly_analysis.nlargest(5, 'Dengesizlik_Orani')[['Dengesizlik_Maliyeti', 'Dengesizlik_Orani', 'PTF']]
print(f"\nEn riskli 5 saat:")
print(risky_hours)

# Mevsimsel risk analizi
seasonal_risk = df.groupby('Ay').agg({
    'Dengesizlik_Maliyeti': ['sum', 'mean', 'std'],
    'Birim_DM': 'mean'
}).round(2)
seasonal_risk.columns = ['Toplam_DM', 'Ortalama_DM', 'DM_Volatilite', 'Birim_DM']
seasonal_risk = seasonal_risk.sort_values('Birim_DM', ascending=False)
print(f"\nMevsimsel Risk Profili (Birim_DM'ye göre büyükten küçüğe sıralı):")
print(seasonal_risk)

# Extreme event analizi
extreme_dm = df[df['Dengesizlik_Maliyeti'] > df['Dengesizlik_Maliyeti'].quantile(0.95)]
print(f"\nExtreme DM Events (>95. persentil):")
print(f"Toplam olay sayısı: {len(extreme_dm)}")
print(f"Bu olayların ortalama PTF'si: {extreme_dm['PTF'].mean():.0f} TL/MWh")
print(f"Bu olayların ortalama dengesizlik büyüklüğü: {extreme_dm['Dengesizlik'].abs().mean():.2f} MWh")

=== MELKOM HES - Risk Analizi ===
En yüksek dengesizlik maliyeti yaşanan 10 gün:
Tarih
2024-11-17     31538.321258
2024-11-10     32031.888363
2024-10-14     77649.830196
2024-10-02     38774.198771
2024-08-31     58886.917697
2024-05-19     35438.989854
2024-05-13     42784.219085
2024-04-07     83980.655164
2024-02-07    103357.217190
2024-02-06     31527.653367
Name: Dengesizlik_Maliyeti, dtype: float64

En riskli 5 saat:
    Dengesizlik_Maliyeti  Dengesizlik_Orani          PTF
22            398.155138              12.15  2415.088192
21            258.407591              11.71  2652.006329
20            238.405860              10.17  2726.846175
19            212.895896               9.82  2727.399399
23            428.852128               9.76  2233.248164

Mevsimsel Risk Profili (Birim_DM'ye göre büyükten küçüğe sıralı):
    Toplam_DM  Ortalama_DM  DM_Volatilite  Birim_DM
Ay                                                 
4   293041.19       407.00        1080.04    150.02
11  29

In [17]:
# En maliyetli 30 saat detay analizi
print("=== MELKOM HES - En Maliyetli 30 Saat Detay Analizi ===")

# En yüksek dengesizlik maliyeti olan 30 saati seç
top_30_cost = df.nlargest(30, 'Dengesizlik_Maliyeti').copy()
top_30_cost['PTF_SMF_Spread'] = abs(top_30_cost['PTF'] - top_30_cost['SMF'])

print(f"En Maliyetli 30 Saat - Genel İstatistikler:")
print(f"Toplam Maliyet: {top_30_cost['Dengesizlik_Maliyeti'].sum():,.0f} TL")
print(f"Ortalama Maliyet: {top_30_cost['Dengesizlik_Maliyeti'].mean():,.0f} TL/saat")
print(f"Toplam Maliyetteki Payı: {top_30_cost['Dengesizlik_Maliyeti'].sum() / df['Dengesizlik_Maliyeti'].sum() * 100:.1f}%")

# Zaman dağılımı
time_distribution = top_30_cost.groupby('Saat').size().sort_values(ascending=False)
print(f"\nEn Maliyetli Saatlerin Zaman Dağılımı:")
print(time_distribution.head())

month_distribution = top_30_cost.groupby('Ay').size().sort_values(ascending=False)
print(f"\nEn Maliyetli Saatlerin Ay Dağılımı:")
print(month_distribution.head())

# Ortak karakteristikler
print(f"\nEn Maliyetli 30 Saatin Karakteristikleri:")
characteristics = top_30_cost.agg({
    'PTF': ['mean', 'min', 'max'],
    'SMF': ['mean', 'min', 'max'],
    'URETIM': ['mean', 'min', 'max'],
    'KGUP': ['mean', 'min', 'max'],
    'Dengesizlik': ['mean', 'min', 'max'],
    'PTF_SMF_Spread': ['mean', 'min', 'max'] if 'PTF_SMF_Spread' in df.columns else ['mean']
}).round(2)

print(characteristics)

# Bu saatlerdeki dengesizlik yönü
positive_imbalance = (top_30_cost['Dengesizlik'] > 0).sum()
negative_imbalance = (top_30_cost['Dengesizlik'] < 0).sum()
print(f"\nDengesizlik Yönü Dağılımı:")
print(f"Pozitif (Fazla Üretim): {positive_imbalance} saat")
print(f"Negatif (Az Üretim): {negative_imbalance} saat")

# Detaylı liste (ilk 10 saat)
print(f"\nEn Maliyetli 10 Saat Detayı:")
detailed_top10 = top_30_cost.head(10)[['Tarih', 'Saat', 'PTF', "SMF", 'KGUP', 'URETIM', 'Dengesizlik', 'Dengesizlik_Maliyeti']]
detailed_top10

=== MELKOM HES - En Maliyetli 30 Saat Detay Analizi ===
En Maliyetli 30 Saat - Genel İstatistikler:
Toplam Maliyet: 239,859 TL
Ortalama Maliyet: 7,995 TL/saat
Toplam Maliyetteki Payı: 9.3%

En Maliyetli Saatlerin Zaman Dağılımı:
Saat
16:00    4
12:00    3
15:00    3
14:00    3
11:00    2
dtype: int64

En Maliyetli Saatlerin Ay Dağılımı:
Ay
10    9
2     6
4     5
5     5
12    2
dtype: int64

En Maliyetli 30 Saatin Karakteristikleri:
          PTF      SMF  URETIM  KGUP  Dengesizlik  PTF_SMF_Spread
mean  1865.84  1423.97    4.98  3.41         1.57         1643.72
min     51.01   300.00    0.00  1.00        -7.00             NaN
max   3000.00  2692.00    7.38  7.00         6.06             NaN

Dengesizlik Yönü Dağılımı:
Pozitif (Fazla Üretim): 20 saat
Negatif (Az Üretim): 10 saat

En Maliyetli 10 Saat Detayı:


,Tarih,Saat,PTF,SMF,KGUP,URETIM,Dengesizlik,Dengesizlik_Maliyeti
2342,2024-04-07,16:00,129.98,2400.00,6.5,1.03,-5.47,12810.8494
6890,2024-10-14,10:00,2888.00,1000.00,1.0,7.04,6.04,11584.7200
2343,2024-04-07,17:00,750.00,2400.00,6.5,0.50,-6.00,10332.0000
3202,2024-05-13,12:00,1313.74,2691.02,7.0,0.00,-7.00,10206.0742
6900,2024-10-14,20:00,2950.00,1367.00,1.0,6.79,5.79,9403.0179
3203,2024-05-13,13:00,1499.66,2691.05,7.0,0.00,-7.00,8904.8505
2340,2024-04-07,14:00,51.01,2400.00,6.5,2.94,-3.56,8618.7244
6891,2024-10-14,11:00,2557.33,1200.00,1.0,7.06,6.06,8443.5798
6902,2024-10-14,22:00,2579.46,675.00,2.4,6.78,4.38,8430.2298
2341,2024-04-07,15:00,81.00,2400.00,6.5,3.00,-3.50,8368.5000


In [10]:
STEP = 1.0

def optimize_kgnp(df, riskli_aylar, riskli_saatler, use_mean=True):
    """
    Belirlenen ay ve saatlere göre KGUP optimizasyonu yapar
    ve özet rapor döner.
    """
    df_opt = df.copy()

    # Saat string → int
    if df_opt['Saat'].dtype == 'object':
        df_opt['Saat'] = pd.to_datetime(df_opt['Saat'], format='%H:%M').dt.hour

    df_opt['KGUP_Opt'] = df_opt['KGUP']

    # --- optimizasyon ---
    for idx, row in df_opt.iterrows():
        if (row['Saat'] in riskli_saatler) and (row['Ay'] in riskli_aylar):
            kgup = row['KGUP_Opt']
            uretim = row['URETIM']
            ptf = row['PTF']
            poz = row['Poz_DF']
            neg = row['Neg_DF']

            d = uretim - kgup  # dengesizlik

            if d < 0:  # eksik üretim
                if neg > ptf:
                    kgup -= STEP
            else:      # fazla üretim
                if poz < ptf:
                    kgup += STEP

            if kgup < 0:
                kgup = 0

            df_opt.at[idx, 'KGUP_Opt'] = kgup

    # --- gelir hesaplamaları ---
    df_opt['GOP_Opt'] = df_opt['KGUP_Opt'] * df_opt['PTF']
    df_opt['Dengesizlik_Opt'] = df_opt['URETIM'] - df_opt['KGUP_Opt']
    df_opt['Fiyat_Opt'] = df_opt.apply(
        lambda row: row['Poz_DF'] if row['Dengesizlik_Opt'] >= 0 else row['Neg_DF'], axis=1
    )
    df_opt['Dengesizlik_Tutarı_Opt'] = df_opt['Dengesizlik_Opt'] * df_opt['Fiyat_Opt']
    df_opt['Net_Opt'] = df_opt['GOP_Opt'] + df_opt['Dengesizlik_Tutarı_Opt']

    # --- toplam sonuç ---
    toplam_mevcut = df['Net_Gelir'].sum()
    toplam_yeni = df_opt['Net_Opt'].sum()
    iyilestirme = toplam_yeni - toplam_mevcut

    print("Mevcut Net Gelir:", toplam_mevcut)
    print("Optimize Net Gelir:", toplam_yeni)
    print("İyileştirme:", iyilestirme)

    # --- detay rapor ---
    ay_ad_tr = {
        1: 'ocak', 2: 'şubat', 3: 'mart', 4: 'nisan', 5: 'mayıs', 6: 'haziran',
        7: 'temmuz', 8: 'ağustos', 9: 'eylül', 10: 'ekim', 11: 'kasım', 12: 'aralık'
    }

    agg_fn = np.mean if use_mean else np.sum
    label = "ort" if use_mean else "top"

    for ay in riskli_aylar:
        for saat in riskli_saatler:
            mask = (df['Ay'] == ay) & (df['Saat'] == saat)
            if mask.sum() == 0:
                continue

            kgup_mevcut = agg_fn(df.loc[mask, 'KGUP'])
            uretim_mevcut = agg_fn(df.loc[mask, 'URETIM'])
            kgup_yeni = agg_fn(df_opt.loc[mask, 'KGUP_Opt'])
            net_mevcut = agg_fn(df.loc[mask, 'Net_Gelir'])
            net_yeni = agg_fn(df_opt.loc[mask, 'Net_Opt'])
            delta_kar = net_yeni - net_mevcut

            print(
                f"ay: {ay_ad_tr.get(ay, ay)} saat: {saat:02d}:00 "
                f"mevcut KGUP({label}): {kgup_mevcut:.2f} "
                f"mevcut uretim({label}): {uretim_mevcut:.2f} "
                f"yeni KGUP({label}): {kgup_yeni:.2f} "
                f"kar_yeni({label}): {net_yeni:,.0f} "
                f"kar_artis({label}): {delta_kar:,.0f}"
            )

    return df_opt

In [12]:
df_opt_risk = optimize_kgnp(df, riskli_aylar=[2, 4, 7, 9, 10, 11], riskli_saatler=[19,20,21,22,23])

Mevcut Net Gelir: 72381318.94182001
Optimize Net Gelir: 72481935.379038
İyileştirme: 100616.43721799552


In [11]:
df_opt_maliyet = optimize_kgnp(df, riskli_aylar=[2, 4, 5, 10, 12], riskli_saatler=[11, 12, 14, 15, 16]) # maliyetli saatlerde KGUP optimizasyonu

Mevcut Net Gelir: 72381318.94182001
Optimize Net Gelir: 72457004.460491
İyileştirme: 75685.51867099106


In [5]:
def optimize_kgnp_combined(df, risk_aylar, risk_saatler, maliyet_aylar, maliyet_saatler):
    """
    Kesişen zamanlar için risk stratejisi, kalanlar için maliyet stratejisi uygular
    """
    df_opt = df.copy()
    
    # Saat string → int
    if df_opt['Saat'].dtype == 'object':
        df_opt['Saat'] = pd.to_datetime(df_opt['Saat'], format='%H:%M').dt.hour
    
    df_opt['KGUP_Opt'] = df_opt['KGUP']
    df_opt['Strateji'] = 'Değişiklik_Yok'
    
    # Risk ve maliyet kümelerini tanımla
    risk_set = set((ay, saat) for ay in risk_aylar for saat in risk_saatler)
    maliyet_set = set((ay, saat) for ay in maliyet_aylar for saat in maliyet_saatler)
    
    # Kesişenleri bul
    kesisen_set = risk_set.intersection(maliyet_set)
    
    # Sadece risk setinde olanlar (kesişenler hariç)
    sadece_risk_set = risk_set - kesisen_set
    
    # Sadece maliyet setinde olanlar (kesişenler hariç)
    sadece_maliyet_set = maliyet_set - kesisen_set
    
    print(f"Kesişen zaman dilimleri: {len(kesisen_set)}")
    print(f"Sadece risk stratejisi: {len(sadece_risk_set)}")
    print(f"Sadece maliyet stratejisi: {len(sadece_maliyet_set)}")
    print(f"Toplam optimize edilecek kombinasyon: {len(kesisen_set) + len(sadece_risk_set) + len(sadece_maliyet_set)}")
    
    # Optimizasyon uygula
    for idx, row in df_opt.iterrows():
        ay_saat = (row['Ay'], row['Saat'])
        
        # Strateji uygulanacak mı kontrol et
        if ay_saat in kesisen_set or ay_saat in sadece_risk_set or ay_saat in sadece_maliyet_set:
            
            # Strateji adını belirle
            if ay_saat in kesisen_set or ay_saat in sadece_risk_set:
                 df_opt.at[idx, 'Strateji'] = 'Risk_Stratejisi'
            else: # sadece_maliyet_set
                 df_opt.at[idx, 'Strateji'] = 'Maliyet_Stratejisi'

            # Ortak optimizasyon mantığını uygula
            kgup = row['KGUP_Opt']
            uretim = row['URETIM']
            ptf = row['PTF']
            poz = row['Poz_DF']
            neg = row['Neg_DF']
            
            d = uretim - kgup
            
            if d < 0:  # eksik üretim
                if neg > ptf:
                    kgup -= 1.0
            else:      # fazla üretim
                if poz < ptf:
                    kgup += 1.0
            
            if kgup < 0:
                kgup = 0
            df_opt.at[idx, 'KGUP_Opt'] = kgup
    
    # Gelir hesaplamaları
    df_opt['GOP_Opt'] = df_opt['KGUP_Opt'] * df_opt['PTF']
    df_opt['Dengesizlik_Opt'] = df_opt['URETIM'] - df_opt['KGUP_Opt']
    df_opt['Fiyat_Opt'] = df_opt.apply(
        lambda row: row['Poz_DF'] if row['Dengesizlik_Opt'] >= 0 else row['Neg_DF'], axis=1
    )
    df_opt['Dengesizlik_Tutarı_Opt'] = df_opt['Dengesizlik_Opt'] * df_opt['Fiyat_Opt']
    df_opt['Net_Opt'] = df_opt['GOP_Opt'] + df_opt['Dengesizlik_Tutarı_Opt']
    
    # Sonuçlar
    toplam_mevcut = df['Net_Gelir'].sum()
    toplam_yeni = df_opt['Net_Opt'].sum()
    iyilestirme = toplam_yeni - toplam_mevcut
    
    print(f"\n=== SONUÇLAR ===")
    print(f"Mevcut Net Gelir: {toplam_mevcut:,.0f} TL")
    print(f"Optimize Net Gelir: {toplam_yeni:,.0f} TL")
    print(f"Toplam İyileştirme: {iyilestirme:,.0f} TL")
    
    # Strateji bazında detay
    strateji_detay = df_opt.groupby('Strateji').agg({
        'Net_Gelir': 'sum',
        'Net_Opt': 'sum'
    })
    strateji_detay['Iyilestirme'] = strateji_detay['Net_Opt'] - strateji_detay['Net_Gelir']
    
    print(f"\n=== STRATEJİ BAZINDA DETAY ===")
    print(strateji_detay)
    
    return df_opt

# Kullanım
df_opt_combined = optimize_kgnp_combined(
    df, 
    risk_aylar=[2, 4, 7, 9, 10, 11], 
    risk_saatler=[19, 20, 21, 22, 23],
    maliyet_aylar=[2, 4, 5, 10, 12], 
    maliyet_saatler=[11, 12, 14, 15, 16]
)

Kesişen zaman dilimleri: 0
Sadece risk stratejisi: 30
Sadece maliyet stratejisi: 25
Toplam optimize edilecek kombinasyon: 55

=== SONUÇLAR ===
Mevcut Net Gelir: 72,381,319 TL
Optimize Net Gelir: 72,557,621 TL
Toplam İyileştirme: 176,302 TL

=== STRATEJİ BAZINDA DETAY ===
                       Net_Gelir       Net_Opt    Iyilestirme
Strateji                                                     
Değişiklik_Yok      5.566509e+07  5.566509e+07       0.000000
Maliyet_Stratejisi  5.417383e+06  5.493069e+06   75685.518671
Risk_Stratejisi     1.129885e+07  1.139946e+07  100616.437218


In [9]:
def kpi_summary(df_monthly, plant_name="Santral"):
    num_cols = df_monthly.select_dtypes(include='number').columns

    summary = pd.DataFrame({
        "SUM": df_monthly[num_cols].sum(),
        "MEAN": df_monthly[num_cols].mean()
    }).round(2)

    summary.index.name = f"{plant_name} - KPI"
    return summary

melkom_summary = kpi_summary(df_monthly, "MELKOM_HES")
melkom_summary

,SUM,MEAN
MELKOM_HES - KPI,,
KGUP,32589.00,2715.75
URETIM,33658.29,2804.86
Dengesizlik,1069.29,89.11
GOP_Geliri,72020149.71,6001679.14
Dengesizlik_Tutarı,361169.24,30097.44
Net_Gelir,72381318.94,6031776.58
Dengesizlik_Maliyeti,2588310.22,215692.52
Dengesizlik_Orani,41.82,3.48
Birim_DM_Aylik,1016.75,84.73


In [11]:
# Dengesizlik performans analizi
print("=== MELKOM HES - Dengesizlik Performans Analizi ===")
print(f"Ortalama Dengesizlik Oranı: {(df['Dengesizlik'].sum() / df['KGUP'].sum() * 100):.2f}%")
print(f"Pozitif Dengesizlik Oranı: {(df['Dengesizlik'] > 0).sum() / len(df) * 100:.1f}%")
print(f"Negatif Dengesizlik Oranı: {(df['Dengesizlik'] < 0).sum() / len(df) * 100:.1f}%")
print(f"Sıfır Üretim Saatleri: {(df['URETIM'] == 0).sum()} saat ({(df['URETIM'] == 0).sum()/len(df)*100:.1f}%)")

# Dengesizlik büyüklük dağılımı
dengesizlik_abs = df['Dengesizlik'].abs()
print(f"\nDengesizlik Büyüklük İstatistikleri:")
print(f"Medyan: {dengesizlik_abs.median():.3f} MWh")
print(f"Ortalama: {dengesizlik_abs.mean():.3f} MWh") 
print(f"95. Persentil: {dengesizlik_abs.quantile(0.95):.3f} MWh")
print(f"Maksimum: {dengesizlik_abs.max():.3f} MWh")

=== MELKOM HES - Dengesizlik Performans Analizi ===
Ortalama Dengesizlik Oranı: 3.28%
Pozitif Dengesizlik Oranı: 46.4%
Negatif Dengesizlik Oranı: 36.0%
Sıfır Üretim Saatleri: 363 saat (4.1%)

Dengesizlik Büyüklük İstatistikleri:
Medyan: 0.580 MWh
Ortalama: 0.973 MWh
95. Persentil: 3.300 MWh
Maksimum: 7.000 MWh


In [15]:
def performance_nonzero(df, name):
    # Sadece üretim yapılan saatleri al
    active = df[df["URETIM"] > 0].copy()
    result = {"Santral": name, "Aktif_Saat": len(active)}

    # Tahmine göre oransal sapma
    valid = active[active["KGUP"] > 0]
    result["Ortalama_Dengesizlik_%"] = (valid["Dengesizlik"].abs() / valid["KGUP"]).mean() * 100
    result["Medyan_Dengesizlik_%"]   = (valid["Dengesizlik"].abs() / valid["KGUP"]).median() * 100

    # Ortalama bias (pozitif = fazla üretim)
    result["Bias_MWh"]     = active["Dengesizlik"].mean()
    result["Pozitif_%"]    = (active["Dengesizlik"] > 0).mean() * 100
    result["Negatif_%"]    = (active["Dengesizlik"] < 0).mean() * 100

    # Dengesizlik büyüklüğü istatistikleri
    result["Medyan_MWh"]   = active["Dengesizlik"].median()
    result["Ortalama_MWh"] = active["Dengesizlik"].mean()
    result["P95_MWh"]      = active["Dengesizlik"].quantile(0.95)
    result["Max_MWh"]      = active["Dengesizlik"].max()

    return result

melkom_perf  = performance_nonzero(df, "MELKOM_HES")

pd.DataFrame([melkom_perf]).round(3)


,Santral,Aktif_Saat,Ortalama_Dengesizlik_%,Medyan_Dengesizlik_%,Bias_MWh,Pozitif_%,Negatif_%,Medyan_MWh,Ortalama_MWh,P95_MWh,Max_MWh
0,MELKOM_HES,8412,34.516,16.036,0.187,48.443,34.391,0.0,0.187,2.8,6.33


In [18]:
# Finansal etki analizi
print("=== MELKOM HES - Finansal Etki Analizi ===")
total_net_revenue = df['Net_Gelir'].sum()
total_dm_cost = df['Dengesizlik_Maliyeti'].sum()
total_production = df['URETIM'].sum()

print(f"Toplam Net Gelir: {total_net_revenue:,.0f} TL")
print(f"Toplam Dengesizlik Maliyeti: {total_dm_cost:,.0f} TL")
print(f"DM'nin Net Gelir'deki Payı: {total_dm_cost/total_net_revenue*100:.2f}%")
print(f"Yıllık Birim Dengesizlik Maliyeti: {total_dm_cost/total_production:.2f} TL/MWh")

# Aylık finansal performans
monthly_financial = df_monthly[['Net_Gelir', 'Dengesizlik_Maliyeti', 'DM_Gelir_Orani']].copy()
print(f"\nEn yüksek DM/Gelir oranına sahip aylar:")
print(monthly_financial.nlargest(3, 'DM_Gelir_Orani'))

# PTF-SMF spread analizi
df['PTF_SMF_Spread'] = abs(df['PTF'] - df['SMF'])
spread_analysis = df.groupby(pd.cut(df['PTF_SMF_Spread'], bins=5)).agg({
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(2)
print(f"\nPTF-SMF Spread'e göre DM analizi:")
spread_analysis

=== MELKOM HES - Finansal Etki Analizi ===
Toplam Net Gelir: 72,381,319 TL
Toplam Dengesizlik Maliyeti: 2,588,310 TL
DM'nin Net Gelir'deki Payı: 3.58%
Yıllık Birim Dengesizlik Maliyeti: 76.90 TL/MWh

En yüksek DM/Gelir oranına sahip aylar:
     Net_Gelir  Dengesizlik_Maliyeti  DM_Gelir_Orani
Ay                                                  
9   2899577.90             175577.59          6.0553
2   4341011.46             255104.80          5.8766
11  5939844.78             293900.03          4.9479

PTF-SMF Spread'e göre DM analizi:


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_8857/3384215808.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spread_analysis = df.groupby(pd.cut(df['PTF_SMF_Spread'], bins=5)).agg({


,Dengesizlik_Maliyeti,Birim_DM
PTF_SMF_Spread,,
"(-2.695, 539.0]",149.45,58.63
"(539.0, 1078.0]",480.97,149.51
"(1078.0, 1617.0]",890.82,246.64
"(1617.0, 2156.0]",1243.41,404.17
"(2156.0, 2695.0]",3456.64,1948.12


In [20]:
# PTF-SMF spread analizi (sadece üretim yapılan saatler)
df_nonzero = df[df['URETIM'] > 0].copy() 

df_nonzero['PTF_SMF_Spread_NonZero'] = abs(df_nonzero['PTF'] - df_nonzero['SMF'])
spread_analysis = df_nonzero.groupby(pd.cut(df_nonzero['PTF_SMF_Spread_NonZero'], bins=5)).agg({
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(2)

print("\nPTF-SMF Spread'e göre DM analizi (sadece üretim yapılan saatler):")
spread_analysis


PTF-SMF Spread'e göre DM analizi (sadece üretim yapılan saatler):


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_8857/2166594354.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spread_analysis = df_nonzero.groupby(pd.cut(df_nonzero['PTF_SMF_Spread_NonZero'], bins=5)).agg({


,Dengesizlik_Maliyeti,Birim_DM
PTF_SMF_Spread_NonZero,,
"(-2.695, 539.0]",145.26,61.35
"(539.0, 1078.0]",469.76,155.26
"(1078.0, 1617.0]",869.77,253.64
"(1617.0, 2156.0]",1254.18,417.21
"(2156.0, 2695.0]",3456.64,1948.12


In [ ]:
def melkom_dengesizlik_yonu_analizi(df):
    """
    MELKOM HES için pozitif vs negatif dengesizlik analizi
    Hangi yönde hata yapmak daha pahalı?
    """
    
    print("=" * 70)
    print("MELKOM HES - POZİTİF vs NEGATİF DENGESİZLİK ANALİZİ")
    print("=" * 70)
    
    # Dengesizlik yönlerini kategorize et
    df_analysis = df.copy()
    df_analysis['dengesizlik_yonu'] = 'Nötr'
    df_analysis.loc[df_analysis['Dengesizlik'] > 0, 'dengesizlik_yonu'] = 'Pozitif_Dengesizlik'  # Fazla ürettik
    df_analysis.loc[df_analysis['Dengesizlik'] < 0, 'dengesizlik_yonu'] = 'Negatif_Dengesizlik'  # Az ürettik
    
    # Her yön için detaylı analiz
    yön_analizi = df_analysis.groupby('dengesizlik_yonu').agg({
        'Dengesizlik': ['count', 'mean', 'std', 'min', 'max'],
        'Dengesizlik_Maliyeti': ['sum', 'mean', 'max'],
        'Birim_DM': ['mean', 'max'],
        'PTF': 'mean',
        'SMF': 'mean',
        'Poz_DF': 'mean',
        'Neg_DF': 'mean'
    }).round(2)
    
    print("\nYÖN BAZLI İSTATİSTİKLER:")
    print("-" * 50)
    
    for yön in ['Pozitif_Dengesizlik', 'Negatif_Dengesizlik', 'Nötr']:
        if yön in yön_analizi.index:
            data = yön_analizi.loc[yön]
            saat_sayisi = data[('Dengesizlik', 'count')]
            toplam_maliyet = data[('Dengesizlik_Maliyeti', 'sum')]
            ort_maliyet = data[('Dengesizlik_Maliyeti', 'mean')]
            
            if yön == 'Pozitif_Dengesizlik':
                print(f"FAZLA ÜRETİM (Tahmin > Gerçek):")
            elif yön == 'Negatif_Dengesizlik':
                print(f"EKSIK ÜRETİM (Tahmin < Gerçek):")
            else:
                print(f"MÜKEMMEL TAHMİN:")
            
            print(f"   Olay Sayısı: {saat_sayisi:.0f} saat")
            print(f"   Toplam Maliyet: {toplam_maliyet:,.0f} TL")
            print(f"   Ortalama Maliyet: {ort_maliyet:.0f} TL/saat")
            print(f"   Ortalama PTF: {data[('PTF', 'mean')]:.0f} TL/MWh")
            print(f"   Ortalama SMF: {data[('SMF', 'mean')]:.0f} TL/MWh")
            print()
    
    # Hangi yön daha maliyetli?
    pos_toplam = df_analysis[df_analysis['dengesizlik_yonu']=='Pozitif_Dengesizlik']['Dengesizlik_Maliyeti'].sum()
    neg_toplam = df_analysis[df_analysis['dengesizlik_yonu']=='Negatif_Dengesizlik']['Dengesizlik_Maliyeti'].sum()
    
    print("KRITIK BULGULAR:")
    print("-" * 30)
    print(f"Pozitif Dengesizlik Maliyeti: {pos_toplam:,.0f} TL")
    print(f"Negatif Dengesizlik Maliyeti: {neg_toplam:,.0f} TL")
    
    if neg_toplam > pos_toplam:
        risk_orani = neg_toplam / pos_toplam
        print(f"EKSIK ÜRETİM {risk_orani:.1f}x DAHA RİSKLİ!")
        print("Çünkü: SMF genelde PTF'den yüksek + %103 ceza çarpanı")
    else:
        risk_orani = pos_toplam / neg_toplam
        print(f"FAZLA ÜRETİM {risk_orani:.1f}x DAHA RİSKLİ!")
        print("Çünkü: PTF düşük dönemlerde fazla üretim zarar")
    
    # Fiyat spread analizi
    print(f"\nFİYAT SPREAD ANALİZİ:")
    print("-" * 30)
    ptf_ort = df['PTF'].mean()
    smf_ort = df['SMF'].mean()
    spread = smf_ort - ptf_ort
    
    print(f"Ortalama PTF: {ptf_ort:.0f} TL/MWh")
    print(f"Ortalama SMF: {smf_ort:.0f} TL/MWh")
    
    if spread > 0:
        print("SMF > PTF olduğu için NEGATİF dengesizlik daha riskli")
        print("   (Eksik üretimde yüksek SMF ile ceza yiyoruz)")
    else:
        print("PTF > SMF olduğu için POZİTİF dengesizlik daha riskli")
    
    # Saatlik risk analizi
    pozitif_saatlik = df_analysis[df_analysis['dengesizlik_yonu']=='Pozitif_Dengesizlik'].groupby(
        df_analysis['Saat'].str.replace(':00', '').astype(int)
    )['Dengesizlik_Maliyeti'].sum().sort_values(ascending=False)
    
    negatif_saatlik = df_analysis[df_analysis['dengesizlik_yonu']=='Negatif_Dengesizlik'].groupby(
        df_analysis['Saat'].str.replace(':00', '').astype(int)
    )['Dengesizlik_Maliyeti'].sum().sort_values(ascending=False)
    
    
    return {
        'pozitif_maliyet': pos_toplam,
        'negatif_maliyet': neg_toplam,
        'risk_yonu': 'negatif' if neg_toplam > pos_toplam else 'pozitif',
        'risk_orani': max(neg_toplam, pos_toplam) / min(neg_toplam, pos_toplam)
    }

dengesizlik_analizi = melkom_dengesizlik_yonu_analizi(df)

MELKOM HES - POZİTİF vs NEGATİF DENGESİZLİK ANALİZİ

YÖN BAZLI İSTATİSTİKLER:
--------------------------------------------------
FAZLA ÜRETİM (Tahmin > Gerçek):
   Olay Sayısı: 4075 saat
   Toplam Maliyet: 1,526,303 TL
   Ortalama Maliyet: 375 TL/saat
   Ortalama PTF: 2240 TL/MWh
   Ortalama SMF: 2160 TL/MWh

EKSIK ÜRETİM (Tahmin < Gerçek):
   Olay Sayısı: 3155 saat
   Toplam Maliyet: 1,062,007 TL
   Ortalama Maliyet: 337 TL/saat
   Ortalama PTF: 2228 TL/MWh
   Ortalama SMF: 2196 TL/MWh

MÜKEMMEL TAHMİN:
   Olay Sayısı: 1545 saat
   Toplam Maliyet: 0 TL
   Ortalama Maliyet: 0 TL/saat
   Ortalama PTF: 2237 TL/MWh
   Ortalama SMF: 2193 TL/MWh

KRITIK BULGULAR:
------------------------------
Pozitif Dengesizlik Maliyeti: 1,526,303 TL
Negatif Dengesizlik Maliyeti: 1,062,007 TL
FAZLA ÜRETİM 1.4x DAHA RİSKLİ!
Çünkü: PTF düşük dönemlerde fazla üretim zarar

FİYAT SPREAD ANALİZİ:
------------------------------
Ortalama PTF: 2235 TL/MWh
Ortalama SMF: 2179 TL/MWh
PTF > SMF olduğu için POZİTİF de

In [26]:
# Pazar dinamikleri analizi
print("=== MELKOM HES - Pazar Dinamikleri Analizi ===")

# Yüksek/düşük fiyat dönemleri
high_price_threshold = df['PTF'].quantile(0.8)
low_price_threshold = df['PTF'].quantile(0.2)

high_price_perf = df[df['PTF'] >= high_price_threshold].agg({
    'URETIM': 'mean',
    'Dengesizlik': 'mean',
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(3)

low_price_perf = df[df['PTF'] <= low_price_threshold].agg({
    'URETIM': 'mean', 
    'Dengesizlik': 'mean',
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(3)

print(f"Yüksek Fiyat Dönemlerinde (PTF > {high_price_threshold:.0f} TL/MWh):")
print(f"Ortalama Üretim: {high_price_perf['URETIM']:.2f} MWh")
print(f"Ortalama Dengesizlik: {high_price_perf['Dengesizlik']:.3f} MWh")
print(f"Ortalama Birim DM: {high_price_perf['Birim_DM']:.2f} TL/MWh")

print(f"\nDüşük Fiyat Dönemlerinde (PTF < {low_price_threshold:.0f} TL/MWh):")
print(f"Ortalama Üretim: {low_price_perf['URETIM']:.2f} MWh")
print(f"Ortalama Dengesizlik: {low_price_perf['Dengesizlik']:.3f} MWh")
print(f"Ortalama Birim DM: {low_price_perf['Birim_DM']:.2f} TL/MWh")

=== MELKOM HES - Pazar Dinamikleri Analizi ===
Yüksek Fiyat Dönemlerinde (PTF > 2742 TL/MWh):
Ortalama Üretim: 3.40 MWh
Ortalama Dengesizlik: 0.195 MWh
Ortalama Birim DM: 65.18 TL/MWh

Düşük Fiyat Dönemlerinde (PTF < 1800 TL/MWh):
Ortalama Üretim: 4.00 MWh
Ortalama Dengesizlik: -0.034 MWh
Ortalama Birim DM: 141.35 TL/MWh


In [8]:
# Kayıp oranı hesabı (mükemmel senaryoya göre)
print("=== MELKOM HES - Kayıp Oranı Hesabı (Mükemmel Senaryo) ===")

# Mükemmel senaryo: Hiç dengesizlik olmasaydı kazanılacak gelir
perfect_revenue = df['URETIM'] * df['PTF']
actual_revenue = df['Net_Gelir']
revenue_loss = perfect_revenue - actual_revenue

# Kayıp oranları
total_perfect_revenue = perfect_revenue.sum()
total_actual_revenue = actual_revenue.sum()
total_revenue_loss = revenue_loss.sum()

print(f"Mükemmel Senaryo Geliri: {total_perfect_revenue:,.0f} TL")
print(f"Gerçek Net Gelir: {total_actual_revenue:,.0f} TL")
print(f"Toplam Gelir Kaybı: {total_revenue_loss:,.0f} TL")
print(f"Kayıp Oranı: {(total_revenue_loss/total_perfect_revenue)*100:.2f}%")

# Aylık kayıp oranları
monthly_loss = df.groupby('Ay').agg({
    'URETIM': 'sum',
    'PTF': lambda x: (df.loc[x.index, 'URETIM'] * df.loc[x.index, 'PTF']).sum(),
    'Net_Gelir': 'sum'
}).round(2)

monthly_loss['Perfect_Revenue'] = df.groupby('Ay').apply(lambda x: (x['URETIM'] * x['PTF']).sum())
monthly_loss['Revenue_Loss'] = monthly_loss['Perfect_Revenue'] - monthly_loss['Net_Gelir']
monthly_loss['Loss_Ratio'] = (monthly_loss['Revenue_Loss'] / monthly_loss['Perfect_Revenue'] * 100).round(2)

print(f"\nAylık Kayıp Oranları:")
print(monthly_loss[['Perfect_Revenue', 'Net_Gelir', 'Revenue_Loss', 'Loss_Ratio']].sort_values(by='Revenue_Loss', ascending=False))

=== MELKOM HES - Kayıp Oranı Hesabı (Mükemmel Senaryo) ===
Mükemmel Senaryo Geliri: 74,969,629 TL
Gerçek Net Gelir: 72,381,319 TL
Toplam Gelir Kaybı: 2,588,310 TL
Kayıp Oranı: 3.45%

Aylık Kayıp Oranları:
    Perfect_Revenue   Net_Gelir  Revenue_Loss  Loss_Ratio
Ay                                                       
11     6.233745e+06  5939844.78   293900.0300        4.71
4      7.537700e+06  7244659.16   293041.1872        3.89
10     6.538680e+06  6257369.84   281310.5085        4.30
2      4.596116e+06  4341011.46   255104.7958        5.55
1      6.356002e+06  6134683.89   221318.4115        3.48
5      9.957199e+06  9747076.43   210122.3896        2.11
8      4.545069e+06  4340217.42   204851.2375        4.51
9      3.075155e+06  2899577.90   175577.5920        5.71
6      7.129240e+06  6954320.86   174919.5654        2.45
3      4.502684e+06  4330656.28   172027.9924        3.82
12     7.724035e+06  7561880.03   162154.9407        2.10
7      6.774002e+06  6630020.89   143981.

/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_1965/1109132416.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_loss['Perfect_Revenue'] = df.groupby('Ay').apply(lambda x: (x['URETIM'] * x['PTF']).sum())


In [36]:
# Over/Under asimetrisi detaylı analizi
print("=== MELKOM HES - Over/Under Asimetrisi Detaylı Analizi ===")

# Pozitif (Over) ve Negatif (Under) dengesizlikleri ayır
over_production = df[df['Dengesizlik'] > 0].copy()
under_production = df[df['Dengesizlik'] < 0].copy()

print(f"Over Production Saatleri: {len(over_production)} ({len(over_production)/len(df)*100:.1f}%)")
print(f"Under Production Saatleri: {len(under_production)} ({len(under_production)/len(df)*100:.1f}%)")

# Over/Under büyüklük analizi
over_stats = {
    'Count': len(over_production),
    'Avg_Size': over_production['Dengesizlik'].mean(),
    'Median_Size': over_production['Dengesizlik'].median(),
    'Max_Size': over_production['Dengesizlik'].max(),
    'Total_Volume': over_production['Dengesizlik'].sum(),
    'Avg_Cost': over_production['Dengesizlik_Maliyeti'].mean(),
    'Total_Cost': over_production['Dengesizlik_Maliyeti'].sum()
}

under_stats = {
    'Count': len(under_production),
    'Avg_Size': under_production['Dengesizlik'].mean(),
    'Median_Size': under_production['Dengesizlik'].median(),
    'Min_Size': under_production['Dengesizlik'].min(),
    'Total_Volume': under_production['Dengesizlik'].sum(),
    'Avg_Cost': under_production['Dengesizlik_Maliyeti'].mean(),
    'Total_Cost': under_production['Dengesizlik_Maliyeti'].sum()
}

asymmetry_df = pd.DataFrame({'Over': over_stats, 'Under': under_stats}).round(3)
print(f"\nOver vs Under Asimetrisi:")
print(asymmetry_df)

# Asimetri oranları
print(f"\nAsimetri Oranları:")
print(f"Hacim Asimetrisi: {abs(over_stats['Total_Volume']) / abs(under_stats['Total_Volume']):.2f}")
print(f"Maliyet Asimetrisi: {over_stats['Total_Cost'] / abs(under_stats['Total_Cost']):.2f}")
print(f"Frekans Asimetrisi: {over_stats['Count'] / under_stats['Count']:.2f}")

=== MELKOM HES - Over/Under Asimetrisi Detaylı Analizi ===
Over Production Saatleri: 4075 (46.4%)
Under Production Saatleri: 3155 (36.0%)

Over vs Under Asimetrisi:
                     Over        Under
Count            4075.000     3155.000
Avg_Size            1.178       -1.183
Median_Size         0.820       -1.000
Max_Size            6.330          NaN
Total_Volume     4801.700    -3732.410
Avg_Cost          374.553      336.611
Total_Cost    1526303.346  1062006.882
Min_Size              NaN       -7.000

Asimetri Oranları:
Hacim Asimetrisi: 1.29
Maliyet Asimetrisi: 1.44
Frekans Asimetrisi: 1.29


In [38]:
# Bias & MAPE gibi tahmin doğruluğu metrikleri
print("=== MELKOM HES - Bias & MAPE Tahmin Doğruluğu Metrikleri ===")

# Sadece üretim yapılan saatleri al
active_hours = df[df['KGUP'] > 0].copy()

# BIAS (Ortalama Hata)
bias = active_hours['Dengesizlik'].mean()
bias_percentage = (bias / active_hours['KGUP'].mean()) * 100

# MAPE (Mean Absolute Percentage Error)
mape = (active_hours['Dengesizlik'].abs() / active_hours['KGUP']).mean() * 100

# MAE (Mean Absolute Error)
mae = active_hours['Dengesizlik'].abs().mean()

# RMSE (Root Mean Square Error)
rmse = (active_hours['Dengesizlik'] ** 2).mean() ** 0.5

# Tahmin doğruluğu (accuracy)
accuracy = 1 - (active_hours['Dengesizlik'].abs().sum() / active_hours['KGUP'].sum())

print(f"BIAS (Ortalama Hata): {bias:.3f} MWh ({bias_percentage:.2f}%)")
print(f"MAPE (Ortalama Mutlak Yüzde Hata): {mape:.2f}%")
print(f"MAE (Ortalama Mutlak Hata): {mae:.3f} MWh")
print(f"RMSE (Kök Ortalama Kare Hata): {rmse:.3f} MWh")
print(f"Tahmin Doğruluğu: {accuracy*100:.1f}%")

# Aylık tahmin performansı
monthly_forecast = active_hours.groupby('Ay').agg({
    'Dengesizlik': ['mean', lambda x: x.abs().mean(), lambda x: (x.abs() / active_hours.loc[x.index, 'KGUP']).mean() * 100],
    'KGUP': 'mean'
}).round(3)

monthly_forecast.columns = ['BIAS', 'MAE', 'MAPE', 'KGUP_Mean']
print(f"\nAylık Tahmin Performansı:")
print(monthly_forecast)

=== MELKOM HES - Bias & MAPE Tahmin Doğruluğu Metrikleri ===
BIAS (Ortalama Hata): 0.121 MWh (3.22%)
MAPE (Ortalama Mutlak Yüzde Hata): 36.50%
MAE (Ortalama Mutlak Hata): 0.983 MWh
RMSE (Kök Ortalama Kare Hata): 1.495 MWh
Tahmin Doğruluğu: 73.9%

Aylık Tahmin Performansı:
     BIAS    MAE    MAPE  KGUP_Mean
Ay                                 
1   0.271  1.126  30.517      4.052
2   0.017  0.893  28.312      3.419
3   0.233  0.846  38.852      2.545
4   0.158  1.119  25.065      5.420
5   0.051  0.780  13.780      6.414
6   0.043  0.991  32.343      4.437
7   0.179  1.022  50.996      3.222
8   0.185  0.965  61.672      2.183
9  -0.060  0.752  41.094      1.810
10  0.208  1.279  54.649      3.980
11  0.028  1.270  40.906      3.508
12  0.127  0.780  21.749      4.074


In [51]:
# Hidro özgü Fiyat Kullanım Endeksi
print("=== MELKOM HES - Hidro Özgü Fiyat Kullanım Endeksi ===")

# Fiyat seviyelerine göre üretim verimliliği (geçici değişkenler kullan)
price_quintiles = df['PTF'].quantile([0.2, 0.4, 0.6, 0.8])
price_levels = pd.cut(df['PTF'], 
                     bins=[0, price_quintiles[0.2], price_quintiles[0.4], price_quintiles[0.6], price_quintiles[0.8], float('inf')],
                     labels=['Çok_Düşük', 'Düşük', 'Orta', 'Yüksek', 'Çok_Yüksek'])

# Her fiyat seviyesinde üretim davranışı
price_efficiency = df.groupby(price_levels).agg({
    'PTF': 'mean',
    'URETIM': ['mean', 'sum', 'count'],
    'KGUP': ['mean', 'sum'],
    'Dengesizlik_Maliyeti': ['mean', 'sum']
}).round(2)

price_efficiency.columns = ['Avg_PTF', 'Avg_Production', 'Total_Production', 'Hour_Count', 'Avg_KGUP', 'Total_KGUP', 'Avg_DM', 'Total_DM']

# Fiyat Kullanım Endeksi hesapla
total_production = df['URETIM'].sum()
weighted_price = (price_efficiency['Total_Production'] * price_efficiency['Avg_PTF']).sum() / total_production
average_price = df['PTF'].mean()
price_utilization_index = weighted_price / average_price

print(f"Fiyat Seviyelerine Göre Üretim Dağılımı:")
print(price_efficiency[['Avg_PTF', 'Total_Production', 'Hour_Count', 'Avg_DM']])

print(f"\nFiyat Kullanım Endeksi: {price_utilization_index:.3f}")
print(f"(1.0'dan yüksek = yüksek fiyatlarda daha çok üretim)")

# Su rejimi stratejisi analizi
daily_production = df.groupby('Tarih').agg({
    'URETIM': 'sum',
    'PTF': 'mean',
    'Dengesizlik_Maliyeti': 'sum'
}).reset_index()

# Günlük üretim vs fiyat korelasyonu
daily_corr = daily_production['URETIM'].corr(daily_production['PTF'])
print(f"\nGünlük Üretim-Fiyat Korelasyonu: {daily_corr:.3f}")

# En değerli günlerde üretim performansı
top_price_days = daily_production.nlargest(30, 'PTF')
avg_production_high_price = top_price_days['URETIM'].mean()
avg_production_overall = daily_production['URETIM'].mean()
high_price_utilization = avg_production_high_price / avg_production_overall

print(f"En Yüksek Fiyatlı 30 Günde Ort. Üretim: {avg_production_high_price:.1f} MWh")
print(f"Genel Ortalama Günlük Üretim: {avg_production_overall:.1f} MWh")
print(f"Yüksek Fiyat Kullanım Oranı: {high_price_utilization:.2f}")

# Hidro strateji skoru
hydro_strategy_score = (price_utilization_index + daily_corr + high_price_utilization) / 3 * 100
print(f"\nHidro Strateji Skoru: {hydro_strategy_score:.1f}/100")
print(f"(Fiyat takibi, günlük korelasyon ve yüksek fiyat kullanımının ortalaması)")

=== MELKOM HES - Hidro Özgü Fiyat Kullanım Endeksi ===
Fiyat Seviyelerine Göre Üretim Dağılımı:
            Avg_PTF  Total_Production  Hour_Count  Avg_DM
PTF                                                      
Çok_Düşük   1161.47           7012.10        1753  382.95
Düşük       2076.96           6255.52        1755  344.08
Orta        2373.99           7038.31        1755  285.68
Yüksek      2629.03           7370.60        1753  264.99
Çok_Yüksek  2939.68           5964.45        1755  197.86

Fiyat Kullanım Endeksi: 0.994
(1.0'dan yüksek = yüksek fiyatlarda daha çok üretim)

Günlük Üretim-Fiyat Korelasyonu: -0.258
En Yüksek Fiyatlı 30 Günde Ort. Üretim: 69.9 MWh
Genel Ortalama Günlük Üretim: 92.0 MWh
Yüksek Fiyat Kullanım Oranı: 0.76

Hidro Strateji Skoru: 49.9/100
(Fiyat takibi, günlük korelasyon ve yüksek fiyat kullanımının ortalaması)


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_8857/2846297821.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_efficiency = df.groupby(price_levels).agg({
